In [1]:
import pandas as pd
import csv
import numpy as np
import operator
from sklearn.neighbors import KNeighborsClassifier
from flask import Flask,render_template,request, redirect, flash
import copy
import yagmail

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [3]:
#data of only islamabad students
isldata = pd.read_csv("islData.csv")
#assigning unique serial nos to students
#isldata['Sr. No'] = isldata.groupby(['Sr. No']).ngroup()

#converting all columns to string
isldata['Warning'] = isldata['Warning'].apply(str)
isldata['CGPA'] = isldata['CGPA'].apply(str)
isldata['SGPA'] = isldata['SGPA'].apply(str)
isldata['Grade Point'] = isldata['Grade Point'].apply(str)
isldata['Credit Hours'] = isldata['Credit Hours'].apply(str)

#course offering csv
#course_offr_df = pd.read_csv("course_offerings.csv")

In [4]:
degree_template = pd.read_csv("completed_degree_template16.csv")
sem_wise_elects = [[0,0],[0,0],[1,0],[2,0],[2,1],[3,2],[3,4],[4,6]]
sem_wise_creds = [17, 34, 50, 67, 85, 100, 115, 130]

# ----------------------------------------------------------------------------------------------------------

# Mapping relationship between courses and their respective labs

In [5]:
course_lab =[
    [2,1], #itc, itc lab
    [6,5], #cp, cp lab
    [8,7], #eng, eng lab I
    [9,10], #dld, dld lab
    [17,16], #coal, coal lab
    [22,23], #os, os lab
    [20,44], #ds, ds lab
    [29,28], #ooad, ooad lab
    [32,33],#cnet, cnet lab
    [35,40]#db, db lab
]

# Dictonary containing all courses a student can study.
It also creates mapping for similar courses that have changed names.
[course_index, core or elective, type of course]

In [6]:
#dictonary of courses where each course is given a unique id
#Also mapping for changed course i.e. ITC and PF
#[Course Code, Core = 1, Elective=0, COurse Type]
course_dict={
"Basic Electronics":[0,1,"EE","EE182"],
'Applied Physics':[0,1,"EE","EE182"],
'Physics - I':[0,1,"EE","EE182"],
"Introduction to Computing Lab":[1,1,"CL","CL101"],
'Introduction to Computing - Lab':[1,1,"CL","CL101"],
"Programming Fundamentals - Lab":[1,1,"CL","CL101"],
"Introduction to Computing":[2,1,"CS","CS101"],
'Programming Fundamentals':[2,1,"CS","CS101"],
"Islamic and Religious Studies":[3,1,"SS","SS111"],
"Calculus and Analytical Geometry":[4,1,"MT","MT101"],
'Calculus - I':[4,1,"MT","MT101"],
'Object Oriented Programming - Lab':[5,1,"CL","CL103"],
"Computer Programming Lab":[5,1,"CL","CL103"],
"Computer Programming":[6,1,"CS","CS103"],
"Object Oriented Programming":[6,1,"CS","CS103"],    
"English Language":[7,1,"SS","SS101"],
"English Composition and Comprehension":[7,1,"SS","SS101"],
"English Language Lab":[8,1,"SL","SL101"],  
'English I - Lab':[8,1,"SL","SL101"],
"English Composition and Comprehension - Lab":[8,1,"SL","SL101"],
"Digital Logic Design":[9,1,"EE","EE227"],
'Computer Logic Design':[9,1,"EE","EE227"],
"Digital Logic Design - Lab":[10,1,"EL","EL227"],
'Computer Logic Design Lab':[10,1,"EL","EL227"],
"Calculus - II":[11,1,"MT","MT115"],
'Differential Equations (Cal II)':[11,1,"MT","MT115"],
'Differential Equations':[11,1,"MT","MT115"],
'Intro to Info. and Comm. Technologies':[12,1,"CL","CL117"],
'Semantic Web Technologies':[13,0,"CS","CS568"],
"Pakistan Studies":[14,1,"SS","SS113"],
'History of Pakistan':[14,1,"SS","SS113"],
"English Composition":[15,1,"SS","SS122"],
'Communication & Presentation Skills':[15,1,"SS","SS122"],
"Comp. Organization and Assembly Lang. Lab":[16,1,"EL","EL213"],
"Comp. Organization and Assembly Lang-Lab":[16,1,"EL","EL213"],
"Comp. Organization and Assembly Lang":[17,1,"EE","EE213"],
"Comp. Organization and Assembly Lang.":[17,1,"EE","EE213"],
"Business Communication - I":[18,1,"SS","SS223"],
"Linear Algebra":[19,1,"MT","MT104"],
"Data Structures":[20,1,"CS","CS201"],
"Discrete Structures":[21,1,"CS","CS211"],
"Operating Systems":[22,1,"CS","CS205"],
"Operating Systems Lab":[23,1,"CL","CL205"],
"Data Mining":[24,0,"CS","CS429"],
"Probability and Statistics":[25,1,"MT","MT206"],
"Theory of Automata":[26,1,"CS","CS301"],
"Computing Methods - I":[27,1,"MT","MT213"],
"Object Oriented Analysis and Design":[28,1,"CS","CS309"],
"Object Oriented Analysis and Design Lab":[29,1,"CL","CL309"],
"Computer Architecture":[30,1,"EE","EE204"],
"Fundamentals of Management":[31,0,"MG","MG223"],
"Computer Networks":[32,1,"CS","CS307"],
"Computer Networks Lab":[33,1,"CL","CL307"],
"Advanced Programming":[34,0,"CS","CS433"],
"Database Systems":[35,1,"CS","CS203"],
"Software Engineering":[36,1,"CS","CS303"],
"Design and Analysis of Algorithms":[37,1,"CS","CS302"],
"Web Programming":[38,0,"CS","CS406"],
"Entrepreneurship":[39,0,"MG","MG414"],
"Database Systems Lab":[40,1,"CL","CL203"],
"Numerical Methods":[41,1,"MT","MT207"],
'Numerical Analysis':[41,1,"MT","MT207"],
'Communication & Presentation Skills - Lab':[42,1,"SL","SL777"],
"Mobile Computing":[43,0,"CS","CS575"],
'Data Structures - Lab':[44,1,"CL","CL218"],
"Technical and Business Writing":[45,1,"SS","SS108"],
"Principles of Marketing":[46,0,"MG","MG220"],
"Marketing Management":[46,0,"MG","MG220"],
'Bio-Informatics':[47,0,"CS","CS508"],
'Introduction to Bioinformatics':[47,0,"CS","CS460"],
'Organizational Behaviour and Leadership':[48,0,"MG","MG340"],
'Organizational Behavior':[48,0,"MG","MG340"],
'Management and Organizational Behavior':[48,0,"MG","MG435"],
'Digital Image Processing':[49,0,"CS","EE402"],
'Psychology':[50,0,"SS","SS118"],
'Advanced Databases':[51,0,"CS","CS502"],
'Network Security':[52,0,"CS","CS411"],
'Management Info. Systems':[53,0,"CS","CS888"],
'Artificial Intelligence':[54,1,"CS","CS401"],
'Software for Mobile Devices':[55,0,"CS","CS440"],
'Project - I':[56,1,"CS","CS491"],
'Professional Issues in IT':[57,1,"CS","CS449"],
'Technical Report Writing':[58,1,"SS","SS108"],
'Project - II':[59,1,"CS","CS492"],
'Sociology':[60,0,"SS","SS127"],
"Marketing Management":[61,0,"MG","MG221"],
'Human Computer Interaction':[62,1,"CS","CS422"],
'Deep Learning for Perception':[63,0,"CS","CS452"],
'Data Warehousing':[64,0,"CS","CS408"],
'Cryptography & Data Security':[65,0,"CS","CS417"],
'Introduction to Block Chain and Cryptocurrency':[66,0,"CS","CS482"],
'Technology Management':[67,0,"MG","MG210"],
'Critical Thinking':[68,0,"SS","SS213"],
'Text Mining':[69,0,"CS","CS987"],
'Software Testing':[70,0,"CS","CS497"],
'Intro. to Software Project Management':[71,0,"CS","CS450"],
'Physics for Engineers':[72,1,"EE","EE555"],
"Conflict Resolution and Negotiation Skills":[73,0,"MG","MG436"],
'Multiagent Systems':[74,0,"CS","CS545"],
'Arabic Language':[75,0,"SS","SS103"],
'Applied Evolutionary Computing':[76,0,"CS","CS443"],
'Remote Sensing and GIS':[77,0,"CS","CS413"],
'Concurrent and Distributed Systems':[77,0,"CS","CS428"],
'Information Retrieval':[78,0,"CS","CS317"],
'Compiler Construction':[79,0,"CS","CS402"],
'Circuit Analysis - I':[80,0,"EE","EE666"],
'Enterprise Information System':[81,0,"CS","CS447"],
'Information Processing Techniques':[82,0,"CS","CS423"],
'User Experience Engineering':[83,0,"CS","CS5107"],
'Computer Graphics':[84,0,"CS","CS418"],
'Information Systems Security':[85,0,"CS","CS446"],
'Fundamentals of Computer Vision':[86,0,"CS","CS495"],
'Introduction to the Internet of Things':[87,0,"CS","CS457"],
'Finance and Accounting':[88,0,"MG","MG103"],
'Accounting and Information System':[89,0,"MG","MG444"],    
'Data Science':[90,0,"CS","CS481"],
'Introduction to Cloud Computing':[91,0,"CS","CS499"],
"Parallel and Distributed Computing":[92,0,"CS","CS416"],
"Natural Language Processing":[93,0,"CS","CS098"],
"Advanced Mobile Application Development":[94,0,"CS","CS464"],
"Cyber Security":[95,0,"CS","CS5101"],
"CS Elective":[96,0,"CS","CS"],
"MG Elective":[97,0,"MG","MG"],
}
#size = 98

# A 2d array for mapping pre_reqs of all courses

In [7]:
#Dependency graph of pre reqs mapped using course dictonary
pre_req = [[1,5], #ITC lab, CP lab
           [1,6], #itc lab, cp
           [2,6], #itc, cp
           [2,5], #itc, cp lab
           [5,20], #cp lab, ds
           [6,20], #cp, ds  #test for only 2 pre req
           [5,44], #cp lab, ds lab
           [6,44], #cp, ds lab
           [20,35], #ds, db
           [20,22], #ds, os
           [20,32], #ds, cnet
           [20,37], #ds, algo
           [20,28], #ds, ooad
           [20,40], #ds, db lab
           [20,23], #ds, os lab
           [20,29], #ds, ooad lab
           [20,33], #ds, cnet lab
           [44,35], #ds lab, db
           [44,22], #ds lab, os
           [44,32], #ds lab, cnet
           [44,37], #ds lab, algo
           [44,28], #ds lab, ooad
           [44,23], #ds lab, os lab
           [44,40], #ds lab, db lab
           [44,33], #ds lab, cnet lab
           [44,29], #ds lab, ooad lab
           [35,51], #db, adv db
           [40,51], #db lab, adv db
           [35,62], #db, hci
           [40,62], #db lab, hci
           [32,52], #cnet, network security
           [33,52], #cnet lab, network security
           [28,36], #ooad, SE
           [29,36], #ooad lab, SE
           #EE
           [9,17], #dld, coal
           [9,16], #dld, coal lab
           [10,17], #dld lab, coal
           [10,16], #dld lab, coal lab
           [17,30], #coal, CA
           [16,30], #coal lab, CA
           #MT
           [4,11], #cal-I, cal-II
           [19,63], #LA, DL
           [19, 49], #LA, DIP
           [21,26], #Discrete Structures, Automata
           #SS
           [7,15], #English Lang, English composition
           [8,15], #English lang lab, Ennglish comp
           [15,58], #English comp, TRW
           [15,45]] # English Comp, TBW


# -----------------------------------------------------------------------------------------------------------

# --------------------Functions start from here----------------------------

# Converting term into numerical terms function: semesters_to_int

In [8]:
#for converting terms(semesters) to numbers
#-1 = summer semester

def semesters_to_int(dframe):
    for i in range (len(dframe)):
        stu_terms = dframe['Semester'].iloc[i].split(' , ')
        temp_term = stu_terms[0]
        term = 1
        sumterm = stu_terms[0]
        nextsumterm = 100
        new_term = []
        for j in range(len(stu_terms)):
            #if "Summer" in stu_terms[j]:
             #   new_term.append(sumterm)
            if temp_term == stu_terms[j]:
                if "Summer" in temp_term:
                    new_term.append(term+.5)
                else:
                    new_term.append(term)  
            
            else:  
                temp_term = stu_terms[j]
                if "Summer" in temp_term:
                    new_term.append(term+.5)
                else:
                    term = term + 1
                    new_term.append(term)  
                
                        
        new_termm = ' , '.join([str(elem) for elem in new_term])
        dframe.Semester.iloc[i] = new_termm
    return dframe
    

In [9]:
#grouping all columns of same student into 1 row
l=isldata.groupby('Sr. No').agg(' , '.join).reset_index()

#Converting semesters to numbers
l = semesters_to_int(l)

M:\Python\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


# course_grade contains courses of all the students in a same manner to make mapping easier.         [course_index, student grade, student cgpa, student warning, how many times he has studied the course]
 

In [10]:
#course_grade is a 2d list. Its [i][0]= serial no of student
#It contains tuples of each students studied course and their relevant grades in that courses. mapped with the dictonary provided above.
#Unstudied courses are represented by -1.
#[course value,grade,cgpa,warning,repeat no.]
def all_stu_course_grade(l):
    course_grade = []
    for k in range (len(l)):
        temp=[]
        for j in range (98):
            temp.append(-1)
        course_grade.append(temp)   
    for i in range(len(l)):
        course_grade[i][0]= l['Sr. No'].iloc[i]
        stu_course_titles=l['Course Title'].iloc[i].split(' , ')
        stu_course_grades=l['Grade Point'].iloc[i].split(' , ')
        stu_course_warning=l['Warning'].iloc[i].split(' , ')
        stu_course_cgpa=l['CGPA'].iloc[i].split(' , ')
        semss = l['Semester'].iloc[i].split(' , ')
        for j in range(0,len(stu_course_grades)):
            index = course_dict[stu_course_titles[j]][0]
            #[index, student grade, student cgpa, student warning, how many times he has studied the course]
            if(course_grade[i][index+1]==-1):
                course_grade[i][index+1]=[index,stu_course_grades[j],stu_course_cgpa[j],stu_course_warning[j],0]    
            else:
                course_grade[i][index+1][1] = stu_course_grades[j]
                course_grade[i][index+1][4]=course_grade[i][index+1][4]+1
                
    return course_grade

# course_type contains information about how many specific types of courses a student has studied and in which semester

In [11]:
def all_stu_course_type(l):
    course_type=[]
    for i in range(len(l)):
        stu_course_titles = l['Course Title'].iloc[i].split(' , ')
        stu_termss = l['Semester'].iloc[i].split(' , ')
        cs_courses = []
        cl_courses = []
        mg_courses = []
        mt_courses = []
        ee_courses = []
        el_courses = []
        ss_courses = []
        sl_courses = []
        for j in range(1,len(stu_course_titles)):
            course = course_dict[stu_course_titles[j]][0]
            typee = course_dict[stu_course_titles[j]][2]
            if typee == "CS":
                cs_courses.append([course,typee,stu_termss[j]])

            if typee == "CL":
                cl_courses.append([course,typee,stu_termss[j]])

            if typee == "EE":
                ee_courses.append([course,typee,stu_termss[j]])

            if typee == "EL":
                el_courses.append([course,typee,stu_termss[j]])

            if typee == "SS":
                ss_courses.append([course,typee,stu_termss[j]])

            if typee == "SL":
                sl_courses.append([course,typee,stu_termss[j]])

            if typee == "MG":
                mg_courses.append([course,typee,stu_termss[j]])

            if typee == "MT":
                mt_courses.append([course,typee,stu_termss[j]])

        c=[]        
        c.append(cs_courses)
        c.append(cl_courses)
        c.append(ee_courses)
        c.append(el_courses)
        c.append(mg_courses)
        c.append(mt_courses)
        c.append(ss_courses)
        c.append(sl_courses)
        course_type.append(c)
    return course_type

# Example students course_grade_array

In [12]:
#It contains tuples of each students studied course and their relevant grades in that courses. mapped with the dictonary provided above.
#Unstudied courses are represented by -1.
#[course value,grade,cgpa,warning,repeat no.]

def ex_stu_course_grade(ex_student):
    ex_course_grade = []

    for j in range (98):
        ex_course_grade.append(-1)

    for i in range(len(ex_student)):
        index = ex_student['Course Title'].iloc[i]
        #[index, student grade, student cgpa, student warning, how many times he has studied the course]
        if(ex_course_grade[index]==-1):
            ex_course_grade[index]=[index,ex_student['Grade Point'].iloc[i],ex_student['CGPA'].iloc[i],ex_student['Warning'].iloc[i],0]    
        else:
            ex_course_grade[index][1] = ex_student['Grade Point'].iloc[i]
            ex_course_grade[index][4] = ex_course_grade[index][4]+1
    return ex_course_grade


# Converting its term to numeirc values. Creating its course_type array and replacing course title names with dictionary values

In [13]:
def ex_stu_course_type(ex_student):
    #mapping specific student wrt course types
    cs_courses = []
    cl_courses = []
    mg_courses = []
    mt_courses = []
    ee_courses = []
    el_courses = []
    ss_courses = []
    sl_courses = []
    
    for j in range(len(ex_student)):
        course = course_dict[ex_student['Course Title'].iloc[j]][0]
        typee = course_dict[ex_student['Course Title'].iloc[j]][2]
        if typee == "CS":
            cs_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "CL":
            cl_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "EE":
            ee_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "EL":
            el_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "SS":
            ss_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "SL":
            sl_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "MG":
            mg_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "MT":
            mt_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])


    stud=[]        
    stud.append(cs_courses)
    stud.append(cl_courses)
    stud.append(ee_courses)
    stud.append(el_courses)
    stud.append(mg_courses)
    stud.append(mt_courses)
    stud.append(ss_courses)
    stud.append(sl_courses)

    #print (cs_courses)

      
    return stud

In [14]:
def semstoNumbers(ex_student):
    #for converting terms(semesters) to numbers
    #-1 = summer semester
    #print (ex_student)
    temp_term = ex_student['Semester'].iloc[0]
    term = 1
    for i in range (len(ex_student)):
        if temp_term == ex_student['Semester'].iloc[i]:
            if "Summer" in temp_term:
                ex_student['Semester'].iloc[i] = term+.5
            else:
                ex_student['Semester'].iloc[i] = term
                                
        else:  
            temp_term = ex_student['Semester'].iloc[i]
            if "Summer" in temp_term:
                ex_student['Semester'].iloc[i] = term+ .5
            else:
                term = term + 1
                ex_student['Semester'].iloc[i] = term  
            

    return ex_student

In [15]:
def crsTitlestoNums(ex_student):
    for i in range(len(ex_student)):
        ex_student['Course Title'].iloc[i] = course_dict[ex_student['Course Title'].iloc[i]][0]
    return ex_student

In [16]:
def crsTitlestoNames(ex_student):
    for i in range(len(ex_student)):
        for j in course_dict.keys():
            if course_dict[j][0] == ex_student['Course Title'].iloc[i]:
                ex_student['Course Title'].iloc[i] = j
    return ex_student

In [17]:
#taking avgs of course types of a student to see which type of course they perfrom better in\
#[CS,CL,EE,EL,MG,MT,SS,SL]
def ex_course_avg(stud):
    averages=[-1,-1,-1,-1,-1,-1,-1,-1]
    for i in range (len(stud)):
        temp_avg = 0
        if(len(stud[i])>0):
            for j in range (len(stud[i])):
                temp_avg = temp_avg + stud[i][j][2]
            averages[i]=temp_avg/len(stud[i])

    return averages

In [18]:
#Coverting all students' semesters equal to the incoming student's semesters

def divide_l_df(total_semesters):
    same_semesters_new = []
    df2 = pd.DataFrame()
    for i in range (len(l)):
        #print (i)
        stu_terms = l['Semester'].iloc[i].split(' , ')  #student's semesters
        stu_cc = l['Course Code'].iloc[i].split(' , ')  #student's course code
        stu_ct = l['Course Title'].iloc[i].split(' , ')  #student's course title
        stu_ch = l['Credit Hours'].iloc[i].split(' , ')  #student's credit hours
        stu_g = l['Grade'].iloc[i].split(' , ')  #student's grade
        stu_gp = l['Grade Point'].iloc[i].split(' , ')  #student's grade point
        stu_sgpa = l['SGPA'].iloc[i].split(' , ')  #student's sgpa
        stu_cgpa = l['CGPA'].iloc[i].split(' , ')  #student's cgpa
        stu_warning = l['Warning'].iloc[i].split(' , ')  #student's warning
 

        rands = stu_terms[len(stu_terms)-1]
        if len(rands) > 1:
            rands = rands[0]
            
        if int(rands)> total_semesters+1:
            same_semesters = []
            same_CourseCode = []
            same_CourseTitle = []
            same_CreditHours = []
            same_Grade = []
            same_GradePoint = [] 
            same_SGPA = []
            same_CGPA = []
            same_Warning = []
            for j in range(len(stu_terms)):
                if float(stu_terms[j])<total_semesters+1:
                    same_semesters.append(stu_terms[j])
                    same_CourseCode.append(stu_cc[j])
                    same_CourseTitle.append(stu_ct[j])
                    same_CreditHours.append(stu_ch[j])
                    same_Grade.append(stu_g[j])
                    same_GradePoint.append(stu_gp[j])
                    same_SGPA.append(stu_sgpa[j])
                    same_CGPA.append(stu_cgpa[j])
                    same_Warning.append(stu_warning[j])


            #same_semesters_new.append(same_semesters)
            same_semesters = ' , '.join([str(elem) for elem in same_semesters])
            same_CourseCode = ' , '.join([str(elem) for elem in same_CourseCode])
            same_CourseTitle = ' , '.join([str(elem) for elem in same_CourseTitle])
            same_CreditHours = ' , '.join([str(elem) for elem in same_CreditHours])
            same_Grade = ' , '.join([str(elem) for elem in same_Grade])
            same_GradePoint = ' , '.join([str(elem) for elem in same_GradePoint])
            same_SGPA = ' , '.join([str(elem) for elem in same_SGPA])
            same_CGPA = ' , '.join([str(elem) for elem in same_CGPA])
            same_Warning = ' , '.join([str(elem) for elem in same_Warning])

            df2= df2.append({"Sr. No":l['Sr. No'].iloc[i], 
                            "Semester":same_semesters,
                               "Course Code":same_CourseCode,
                               "Course Title":same_CourseTitle,
                               "Credit Hours":same_CreditHours,
                               "Grade":same_Grade,
                               "Grade Point":same_GradePoint,
                               "SGPA":same_SGPA,
                               "CGPA":same_CGPA,
                               "Warning":same_Warning},ignore_index=True)
        
    return df2

# In must_take all F courses that are present in course offering are added


In can_take all courses of course offering except for previously studied pre reqs are added

In can_take_repeat courses that are present in the student transcrpt as well as the curse offering are added

In can_take_new all new courses from course offering are added

In [19]:
def ex_suggest_courses(ex_student,ex_course_grade,crsOffer):
    #in case of summer semester
    stu_offer_title = ex_student['Course Title'].tolist()

    offer_title = crsOffer['Course Title'].tolist()
     
    can_take_courses = []
    #for taking all courses
    can_take_courses_repeat = []
    can_take_courses_new = []
    #for handling only F and W cases 
    must_take_courses = []
    prereq_remove = []

    #no_of_courses = input("How many courses do you want to take in Summer? ")

    for i in range(len(ex_course_grade)):
        if (ex_course_grade[i] != -1):
            if (ex_course_grade[i][1]=='0.0'):
                for d in range(len(course_lab)):
                    #applying the advisory rule that if student fails course then its lab is must take
                    if ex_course_grade[i][0] == course_lab[d][0]:
                        must_take_courses.append(course_lab[d][1])
                must_take_courses.append(ex_course_grade[i][0])
                #can_take_courses.append(ex_course_grade[i][0])

    must_take_courses = list(set(must_take_courses))
    remove_courses = []
    for i in range (len(must_take_courses)):
        if must_take_courses[i] not in offer_title:
            remove_courses.append(must_take_courses[i])


    for i in range(len(must_take_courses)):
         for k in course_dict.keys():
            if course_dict[k][0] == must_take_courses[i]:
                if course_dict[k][1] == 0:
                    remove_courses.append(must_take_courses[i])

    remove_courses = list(set(remove_courses))

    for i in range(len(remove_courses)):
        must_take_courses.remove(remove_courses[i])

    must_take_courses = list(dict.fromkeys(must_take_courses))
    
    #------------------------------------------------------------

    #Filling can_take with courses from course offering except for pre req
    can_take_courses = offer_title

    #find studied preqs in transcript
    for i in range(len(can_take_courses)):
        for j in range(len(pre_req)):
            if pre_req[j][1] == can_take_courses[i]:
                prereq_remove.append(pre_req[j][0])
        
    rem_same = []
    for i in range(len(can_take_courses)):
        if can_take_courses[i] in must_take_courses:
            print("something is added")
            rem_same.append(can_take_courses[i])

    for i in range(len(rem_same)):
        can_take_courses.remove(rem_same[i])
    #removing prereqs from can_take_course
    prereq_remove = list(dict.fromkeys(prereq_remove))    
    for i in range(len(prereq_remove)):
        if prereq_remove[i] in can_take_courses_repeat:
            can_take_courses.remove(prereq_remove[i])


    #repeat courses only
    for i in range(len(can_take_courses)):
        if can_take_courses[i] in stu_offer_title:
            can_take_courses_repeat.append(can_take_courses[i])
        else:
            can_take_courses_new.append(can_take_courses[i])

    #looking at the tendency of student

    #--------------------------------------------------------


    can_take_courses_string = []
    for q in range(len(can_take_courses)):
        for k in course_dict.keys():
            if course_dict[k][0] == can_take_courses[q]:
                can_take_courses_string.append(k)
                break
    can_take_courses_repeat_string = []
    for q in range(len(can_take_courses_repeat)):
        for k in course_dict.keys():
            if course_dict[k][0] == can_take_courses_repeat[q]:
                can_take_courses_repeat_string.append(k)
                break


    can_take_courses_new_string = []
    for q in range(len(can_take_courses_new)):
        for k in course_dict.keys():
            if course_dict[k][0] == can_take_courses_new[q]:
                can_take_courses_new_string.append(k)
                break

    must_take_courses_string = []
    for q in range(len(must_take_courses)):
        for k in course_dict.keys():
            if course_dict[k][0] == must_take_courses[q]:
                must_take_courses_string.append(k)
                break


    return can_take_courses, can_take_courses_string, can_take_courses_new, can_take_courses_new_string, can_take_courses_repeat, can_take_courses_repeat_string, must_take_courses, must_take_courses_string


In [20]:
#converting to numeric values of Course Title
def knn_pred_courses(fil_crs, musttake,course_offr_df):
    
    #knn_stu_crse = fil_crs
    knn_stu_crse = copy.copy(fil_crs)
    
    crss = len(fil_crs)- len(musttake)    
        
    for i in range(len(course_offr_df)):
        for j in range(len(fil_crs)):
            if int(course_offr_df['Course Title'].iloc[i]) == fil_crs[j]:
                knn_stu_crse[j]= [fil_crs[j],float(course_offr_df['Credit Hours'].iloc[i])]
                print(knn_stu_crse[j],float(course_offr_df['Credit Hours'].iloc[i]))
                
                
    return knn_stu_crse

In [21]:
def warning_cgpa_range(array,rangeee):
    for i in range(len(array)):
        if array[i][1] >= 4.0:
            rangeee.append([3.67,4.0])
        elif array[i][1] <= 0.0:
            rangeee.append([0.00,0.00])
        else:
            rangeee.append([array[i][1] -.33 ,array[i][1] + .33])

    return rangeee


In [22]:
def cgpa_range(array,rangeee):
    for i in range(len(array)):
        if array[i][1] >= 4.0:
            rangeee.append([3.67,4.0])
        elif array[i][1] <= 0.0:
            rangeee.append([0.00,0.00])
        else:
            rangeee.append([array[i][1] ,array[i][1] + .67])

    return rangeee


In [23]:
def gpa_calculate(type_gpa,rang,knn_stu_crse):
    ttl_crdit = 0
    sumgp = 0
    for i in range(len(rang)):
        sumgp = sumgp + knn_stu_crse[i][2]* rang[i][type_gpa]
        ttl_crdit = ttl_crdit + knn_stu_crse[i][2]

    gpa = sumgp/ttl_crdit
    
    if gpa > 4.0:
        gpa = 4.0
    return gpa


In [24]:
def musheer_suggestion(knn_stu_crse,range3):
    musheer_given_courses = []
    print("knn crsesss in musheer suggestion:", knn_stu_crse)
    for i in range(len(range3)):
        musheer_given_courses.append(-1)
    for j in range(len(knn_stu_crse)):
        for i in course_dict.keys():
            if course_dict[i][0] == knn_stu_crse[j][0]:
                musheer_given_courses[j] = [i, str(range3[j][0]) + " to " + str(range3[j][1])]
                break
    musheer_given_courses = pd.DataFrame(musheer_given_courses)
    musheer_given_courses.rename(columns = {0:"Courses",1:"Tentative Grades"},inplace = True)
    
    return musheer_given_courses

In [25]:
def student_choice_courses(stu_choose_self,course_offr_df,g_a):
    for i in range(len(course_offr_df)):
            for j in range(len(stu_choose_self)):
                if int(course_offr_df['Course Title'].iloc[i]) == stu_choose_self[j]:
                    stu_choose_self[j] = [stu_choose_self[j],g_a[j],course_offr_df['Credit Hours'].iloc[i]]
    return stu_choose_self   

In [26]:
def course_suggestionnn(ex,degree_template,sem, this_creds):
    template_crses = []
          
    if sem == 9:
        template_crses = [56, 54, 62, 96, 96]
        
    if sem == 10:
        template_crses = [59, 57, 96, 96, 97]
        
    if sem == 8:
        template_crses = [97, 97, 96, 36, 41]
        
    if sem == 7:
        template_crses =[28, 29, 32 ,33, 26, 96, 25]
        
        
    return template_crses

In [27]:
#to check whether a student has normal degree or not
def normal_template_sem(ex, musttake, degree_template, credlimit, course_offring, cs_elects, mg_elects, semNum):
    template_crses = []
    template_crses = degree_template[degree_template['Semester'] == ex["Semester"].iloc[len(ex)-1]+1]['Course Title']
    
    template_crses = list(template_crses)
    credits = 0
    for i in range(len(template_crses)):
        template_crses[i]= course_dict[template_crses[i]][0]
    
    for i in range(len(course_offring)):
        for j in range(len(template_crses)):
            if int(course_offring['Course Title'].iloc[i]) == template_crses[j]:
                credits = credits + course_offring['Credit Hours'].iloc[i]
    
    
    print("Credits in normal:",credits)
    
    
    # check if course offering names in names or numbers
    musttake_creds = 0
    for i in range(len(course_offring)):
        for j in range(len(musttake)):
            if int(course_offring['Course Title'].iloc[i]) == musttake[j]:
                musttake_creds = musttake_creds + course_offring['Credit Hours'].iloc[i]
    
    
    #find cred hours of must take 
    replacee = []
    if len(musttake) != 0:
        if credits + 3 > credlimit:
            for i in range(len(template_crses)):
                for j in course_dict.keys():
                    if course_dict[j][0] == template_crses[i]:
                        if course_dict[j][1] == 0:
                            replacee.append(template_crses[i])
        else:
            for i in range(len(musttake)):
                template_crses.append(musttake[i])
     
    print("replacee",replacee)
    if len(replacee) != 0:
        check = 0
        for i in range(len(replacee)):
            for j in range(len(musttake)):
                if check == len(musttake):
                    break
                
                else:
                    template_crses.remove(replacee[i])
                    template_crses.append(musttake[j])
                    check = check + 1
                    
            if check == len(musttake):
                break     
                
    credits = 0
    for i in range(len(course_offring)):
        for j in range(len(template_crses)):
            if int(course_offring['Course Title'].iloc[i]) == template_crses[j]:
                credits = credits + course_offring['Credit Hours'].iloc[i]
    
    
    cs_count = 0
    mg_count = 0
    for i in template_crses:
        if i == 96:
            cs_count = cs_count + 1
        if i == 97:
            mg_count = mg_count + 1
            
        
    if credits + 3 <= credlimit:
        print("mg elects",mg_elects)
        print("mg count", mg_count)
        if mg_elects + mg_count != sem_wise_elects[semNum -1][0]:                
            template_crses.append(97)
            credits = credits + 3
            
    if credits + 3 <= credlimit:
        
        if cs_elects + cs_count != sem_wise_elects[semNum -1][1]:
            template_crses.append(96)
    
    return template_crses        

In [28]:
def min_grade_to_remove_warning(stu_choose_self,stu_cgpa,ex_student):    
    credits_earned = 0
    to_earn_credits = 0
    stu_offer_title = ex_student['Course Title'].tolist()
    u_stu_offer_title = list(dict.fromkeys(stu_offer_title))
    for i in range(len(u_stu_offer_title)):
        for j in range(len(ex_student)):
            if u_stu_offer_title[i] == ex_student['Course Title'].iloc[j]:
                credits_earned = credits_earned + float(ex_student['Credit Hours'].iloc[j])
                break
                
    print("stu choose self in min grade warning:", stu_choose_self)
    for i in range(len(stu_choose_self)):
        for j in range(len(ex_student)):
            if stu_choose_self[i][0] == ex_student['Course Title'].iloc[j]:
                to_earn_credits = to_earn_credits + float(ex_student['Credit Hours'].iloc[j])
                #print(to_earn_credits)
                break

                    
    for i in range(len(stu_choose_self)):        
        to_earn_credits = to_earn_credits + stu_choose_self[i][1]
        #print(to_earn_credits)

    prev_thing = 0.0
    credit_earnedd = 0

    for i in range(len(ex_student)):
        if float(ex_student['Course Title'].iloc[i])  not in stu_choose_self:
            prev_thing = prev_thing + (float(ex_student['Credit Hours'].iloc[i])*float(ex_student['Grade Point'].iloc[i]))
            credit_earnedd = credit_earnedd + float(ex_student['Credit Hours'].iloc[i])

    x =(2*(credits_earned+to_earn_credits) - prev_thing)/to_earn_credits
    
    return x

In [29]:
def student_given_courses(stu_choose_self,rangee1):
    self_given_courses = []
    for j in range(len(stu_choose_self)):
        for i in course_dict.keys():
            if course_dict[i][0] == stu_choose_self[j][0]:
                self_given_courses.append( i +"   "+str(rangee1[j][0]) + " - " + str(rangee1[j][1]))
    return self_given_courses


In [30]:
def gradePoint_to_letterGrade(rangee):
    rangee_gr=copy.copy(rangee)
    for i in range(len(rangee)):
        if round(rangee[i][0],2) == 0.00:
            rangee_gr[i][0] = 'F'
            
        elif round(rangee[i][0],2) == 0.67 or round(rangee[i][0],2) == 0.66:
            rangee_gr[i][0] = 'D'
            
        elif round(rangee[i][0],2) == 1.00:
            rangee_gr[i][0] = 'D'
            
        elif round(rangee[i][0],2) == 1.33 or round(rangee[i][0],2) == 1.34:
            rangee_gr[i][0] = 'D+'
            
        elif round(rangee[i][0],2) == 1.67 or round(rangee[i][0],2) == 1.66:
            rangee_gr[i][0] = 'C-'
            
        elif round(rangee[i][0],2) == 2.00:
            rangee_gr[i][0] = 'C'
            
        elif round(rangee[i][0],2) == 2.33 or round(rangee[i][0],2) == 2.34:
            rangee_gr[i][0] = 'C+'
            
        elif round(rangee[i][0],2) == 2.67 or round(rangee[i][0],2) == 2.66:
            rangee_gr[i][0] = 'B-'
            
        elif round(rangee[i][0],2) == 3.00:
            rangee_gr[i][0] = 'B'
            
        elif round(rangee[i][0],2) == 3.33 or round(rangee[i][0],2) == 3.22 or round(rangee[i][0],2) == 3.34:
            rangee_gr[i][0] = 'B+'
            
        elif round(rangee[i][0],2) == 3.67 or round(rangee[i][0],2) == 3.66:
            rangee_gr[i][0] = 'A-'
            
        elif round(rangee[i][0],2) >= 4.00 or round(rangee[i][0],2) == 4.34 or round(rangee[i][0],2) == 3.89 or round(rangee[i][0],2) == 3.90 or round(rangee[i][0],2) == 3.80:
            rangee_gr[i][0] = 'A'
            
            
        if round(rangee[i][1],2) == 0.00:
            rangee_gr[i][1] = 'F'
            
        elif round(rangee[i][1],2) == 0.67 or round(rangee[i][1],2) == 0.66:
            rangee_gr[i][1] = 'D'
            
        elif round(rangee[i][1],2) == 1.00:
            rangee_gr[i][1] = 'D'
            
        elif round(rangee[i][1],2) == 1.33 or round(rangee[i][1],2) == 1.34:
            rangee_gr[i][1] = 'D+'
            
        elif round(rangee[i][1],2) == 1.67 or round(rangee[i][1],2) == 1.66:
            rangee_gr[i][1] = 'C-'
            
        elif round(rangee[i][1],2) == 2.00:
            rangee_gr[i][1] = 'C'
            
        elif round(rangee[i][1],2) == 2.33 or round(rangee[i][1],2) == 2.34:
            rangee_gr[i][1] = 'C+'
            
        elif round(rangee[i][1],2) == 2.67 or round(rangee[i][1],2) == 2.66:
            rangee_gr[i][1] = 'B-'
            
        elif round(rangee[i][1],2) == 3.00:
            rangee_gr[i][1] = 'B'
            
        elif round(rangee[i][1],2) == 3.33 or round(rangee[i][1],2) == 3.34:
            rangee_gr[i][1] = 'B+'
            
        elif round(rangee[i][1],2) == 3.67 or round(rangee[i][1],2) == 3.66:
            rangee_gr[i][1] = 'A-'
            
        elif round(rangee[i][1],2) >= 4.00 or round(rangee[i][1],2) == 4.34 or round(rangee[i][1],2) == 3.89 or round(rangee[i][1],2) == 3.90 or round(rangee[i][1],2) == 3.80:
            rangee_gr[i][1] = 'A'
            
    return rangee_gr
   

In [31]:
def example_student(example):
    #example students course grade array
    ex_course_grades = ex_stu_course_grade(example)
    
    can_take_courses = []
    can_take_courses_string = []
    can_take_courses_new = []
    can_take_courses_new_string = []
    can_take_courses_repeat = []
    can_take_courses_repeat_string = []
    must_take_courses = []
    must_take_courses_string = []
    
    all_terms = example['Semester'].unique()
    semNum = example['Semester'].iloc[len(example)-1]+1
    if semNum == 0:
        semNum = all_terms[len(all_terms)-2]
        
    if len(str(semNum))>2:
        a =str(semNum)
        
        if int(a[0]) == 9 or int(a[0]) == 11 or int(a[0]) == 13 or int(a[0]) == 15:
            semFile = "Semester7"+ ".csv"
        else:
            semFile = "Semester" +a[0]  + ".csv"
        
    else:
        if semNum == 10 or semNum == 12 or semNum == 14 or semNum == 16:
            semFile = "Semester8" + ".csv"
            
        if semNum == 9 or semNum == 11 or semNum == 13 or semNum == 15:
            semFile = "Semester7" + ".csv"
        else:   
            semFile = "Semester" + str(semNum) + ".csv"
        
    print("Ab is semester ki cheez dhundani hai:",semNum)
    co = pd.read_csv(semFile)
    co = crsTitlestoNums(co)
    can_take_courses, can_take_courses_string, can_take_courses_new, can_take_courses_new_string, can_take_courses_repeat, can_take_courses_repeat_string, must_take_courses, must_take_courses_string = ex_suggest_courses(example,ex_course_grades,co)

    return ex_course_grades, example, can_take_courses, can_take_courses_string, can_take_courses_new, can_take_courses_new_string, can_take_courses_repeat, can_take_courses_repeat_string, must_take_courses, must_take_courses_string,co


In [32]:
def warning2_musheerSuggestion(student, filtered_courses,musttake_courses, s_c_o,ex_cg,sgpas):
    suggested_crses = []
    for i in range(len(must_take_courses)):
        suggested_crses.append(must_take_courses[i])
        
    crs_count = len(musttake_courses)
    if crs_count < 4:
        semesters = student['Semester'].iloc[len(student)-1]
        if len(str(semesters)) > 1:
            semesters = student['Semester'].iloc[len(student)-2]
        last_sem = student[student['Semester'] == semesters]
            
        min_grade = 1.0
        while crs_count < 4:
            for i in range(len(last_sem)):
                if last_sem['Grade Point'].iloc[i] == str(min_grade):
                    if last_sem['Course Title'].iloc[i] in filtered_courses:
                        suggested_crses.append(last_sem['Course Title'].iloc[i])
                        crs_count = crs_count + 1
                if crs_count == 4:
                    break
                            
    #suggested_crses = list(set(suggested_crses))   
    print("in warning 2222",suggested_crses)
    knn_stu_crs = knn_pred_courses(suggested_crses,musttake_courses,s_c_o)
    
    if(len(ex["CGPA"].iloc[len(ex)-1])<5):
        gpaaa = float(ex["CGPA"].iloc[len(ex)-1])
    else:
        gpaRange = ex["CGPA"].iloc[len(ex)-1][:4]
        gpaaa = float(gpaRange)
    
    minG_rmve_warning = min_grade_to_remove_warning(knn_stu_crs, gpaaa,student)
    
    g_a =Warning_assign_Grades(suggested_crses,avgs,ex_cg,minG_rmve_warning)
    
    knn_stu_crs = add_grade_in_arr(knn_stu_crs, g_a)
    
    
    rangee = []
    rangee = warning_cgpa_range(knn_stu_crs,rangee)
    mingpa = gpa_calculate(0,rangee,knn_stu_crs)
    maxgpa = gpa_calculate(1,rangee,knn_stu_crs)
    rangee1 = gradePoint_to_letterGrade(rangee)
    musheer_given_courses = musheer_suggestion(knn_stu_crs,rangee1)
    
  
    cgpa_musheer = str(round(mingpa,2)) + " - " + str(round(maxgpa,2))

    new_sem_df_ = pd.DataFrame()
    new_sem_df_ = newSem_df(ex, rangee1,knn_stu_crs,mingpa,maxgpa,sgpas,semesters)
    return musheer_given_courses, cgpa_musheer, new_sem_df_,round(maxgpa,2)

In [51]:
def filter_suggestions(ex, selected_courses, musttake_courses, cantake,credit_limit,course_offring,cs_elects, mg_elects, degree_template,sem, creds):
    filtered_crses = []
    this_creds = 0
    #knn_crses = []
    #knn_crses = degree_template[degree_template['Semester'] == ex["Semester"].iloc[len(ex)-1]+1]['Course Title']
            
        #else:
             #knn_crses = selected_courses[0]['Course Title']
    #else:
         #knn_crses = selected_courses[0]['Course Title']
    
        
    
    #knn_crses = list(knn_crses)
    
    filtered_crses = normal_template_sem(ex,must_take_courses, degree_template, 18, course_offring,cs_elects,mg_elects,sem)
    print("suggestions in filter_sugs",filtered_crses)
    #print("knn in filter suggestionL", knn_crses)
        
    #for i in range(len(knn_crses)):
        #knn_crses[i]= course_dict[knn_crses[i]][0]
       
        
    #filtered_crses = []
    #for i in knn_crses:
     #   if i in cantake:
      #      filtered_crses.append(i)
            
            
    #for i in range(len(musttake_courses)):    
        #filtered_crses.append(musttake_courses[i])
        
        
    #credits = 0
    #for i in range(len(course_offring)):
     #   for j in range(len(filtered_crses)):
      #      if int(course_offring['Course Title'].iloc[i]) == filtered_crses[j]:
       #         credits = credits + course_offring['Credit Hours'].iloc[i]
        
    

    #to_remove = []
    #for i in range(len(filtered_crses)):
     #   if credits>credit_limit:
      #      for k in course_dict.keys():
       #         if course_dict[k][0] == filtered_crses[i]:
        #            if course_dict[k][1] == 0:
         #               to_remove.append(filtered_crses[i])
        #else:
         #   break
                        
   
    #to_remove = list(set(to_remove))
    #for i in range(len(to_remove)):
     #   filtered_crses.remove(to_remove[i])
            
    return filtered_crses

In [52]:
def cgpa_calculate(ex, mingpa,maxgpa,sgpas,warning):
    minsumm = 0
    maxsumm = 0
    #warning = 0 
    for i in range(len(sgpas)):
        if len(sgpas[i])<5:
            minsumm =minsumm + float(sgpas[i])
            maxsumm =maxsumm + float(sgpas[i])
        else:
            minsumm =minsumm + float(sgpas[i][:4])
            maxsumm =maxsumm + float(sgpas[i][7:])
            
    
    min_gpa = (minsumm + mingpa)/(len(sgpas)+1)
    max_gpa = (maxsumm + maxgpa)/(len(sgpas)+1)
    
    if max_gpa <=2.0:
        warning = float(ex["Warning"].iloc[len(ex)-1]) +1
    
    return str(round(min_gpa,2)) + " - " + str(round(max_gpa,2)),min_gpa, warning

In [53]:
def newSem_df(ex,grade_range, course_array,mingpa,maxgpa,sgpas,ts):
    new_sem_list = []
    warning = 0
    for i in range(len(course_array)):
        new_sem_list.append([])
        for j in range(10):
            new_sem_list[i].append(-1)
        
    cgpaa,min_sgpa,warning = cgpa_calculate(ex, mingpa,maxgpa,sgpas,warning)    
    for j in range(len(course_array)):
        for i in course_dict.keys():
            if course_dict[i][0] == course_array[j][0]:
                new_sem_list[j][1] = ts + 1
                new_sem_list[j][3] = i
                new_sem_list[j][2] = course_dict[i][3]
                new_sem_list[j][4] = course_array[j][2]
                new_sem_list[j][5] = str(grade_range[j][0]) + " to " +str(grade_range[j][1])
                new_sem_list[j][9] = warning
                
        new_sem_list[j][7] = str(round(mingpa,2)) + " - " + str(round(maxgpa,2))
        new_sem_list[j][8] = cgpaa            
        
                
    nsdf = pd.DataFrame(new_sem_list)
    nsdf.rename(columns = {0:'Sr. No', 1:'Semester', 
                              2:'Course Code',3:'Course Title',4:'Credit Hours',5:'Grade',6:'Grade Point',7:'SGPA',8:'CGPA',9:'Warning'}, inplace = True)
    
    nsdf['Warning'] = nsdf['Warning'].apply(str)
    nsdf['CGPA'] = nsdf['CGPA'].apply(str)
    nsdf['SGPA'] = nsdf['SGPA'].apply(str)
    nsdf['Grade Point'] = nsdf['Grade Point'].apply(str)
    nsdf['Credit Hours'] = nsdf['Credit Hours'].apply(str)
    
    return nsdf

In [54]:
def crse_by_type(crse_type,avgs):
    if crse_type == "CS":
        gradee = grade_by_avg(avgs[0])

    elif crse_type == "CL":
        gradee = grade_by_avg(avgs[1])
        
    elif crse_type == "MG":
        gradee = grade_by_avg(avgs[4])
        
    elif crse_type == "MT":
        gradee = grade_by_avg(avgs[5])
        
    elif crse_type == "EE":
        gradee = grade_by_avg(avgs[2])
        
    elif crse_type == "EL":
        gradee = grade_by_avg(avgs[3])
        
    elif crse_type == "SS":
        gradee = grade_by_avg(avgs[6])
        
    elif crse_type == "SL":
        gradee = grade_by_avg(avgs[7])
    
    return gradee

In [55]:
def grade_by_avg(avg):
    gradee = 0.0
    if avg>3.67 and avg<= 4.00:
        gradee = 4.00
        
    elif avg>4.00:
        gradee = 4.00
        
    elif avg>3.33 and avg<= 3.67:
        gradee = 3.67
        
    elif avg>3.00 and avg<= 3.33:
        gradee = 3.33
        
    elif avg>2.67 and avg<= 3.00:
        gradee = 3.00
        
    elif avg>2.33 and avg<= 2.67:
        gradee = 2.67
        
    elif avg>2.00 and avg<= 2.33:
        gradee = 2.33
        
    elif avg>1.67 and avg<= 2.00:
        gradee = 2.00
        
    elif avg>1.33 and avg<= 1.67:
        gradee = 1.67
        
    elif avg>1.00 and avg<= 1.33:
        gradee = 1.33
        
    elif avg>0.0 and avg<= 1.00:
        gradee = 1.00
        
    elif avg<=0.0:
        gradee = 0

    return gradee

In [56]:
def assign_Grades(crses, avgs, ecga,min_grade_thresh):
    #crses = list of courses thats it
    #avgs = [cs_courses, cl_courses, ee_courses, el_courses, mg_courses, mt_courses, ss_courses, sl_courses]
    #[course value,grade,cgpa,warning,repeat no.]
    
    grade_arr = []
    for i in range (len(crses)):
        for j in course_dict.keys():
            if course_dict[j][0] == crses[i]:
                crse_type = course_dict[j][2]
                gradee = crse_by_type(crse_type,avgs)
                
                
                if ecga[crses[i]]!=-1:
                    grade_arr.append(gradee+1)
                else:
                    if gradee< min_grade_thresh:
                        if course_dict[j][1] == 0:
                            gradee = min_grade_thresh
                
                    grade_arr.append(gradee)
                break
    return grade_arr

In [57]:
def Warning_assign_Grades(crses, avgs, ecga,min_grade_thresh):
    #crses = list of courses thats it
    #avgs = [cs_courses, cl_courses, ee_courses, el_courses, mg_courses, mt_courses, ss_courses, sl_courses]
    #[course value,grade,cgpa,warning,repeat no.]
    
    grade_arr = []
    for i in range (len(crses)):
        for j in course_dict.keys():
            if course_dict[j][0] == crses[i]:
                crse_type = course_dict[j][2]
                gradee = crse_by_type(crse_type,avgs)
                
                grade_arr.append(gradee)
                break
    return grade_arr

In [58]:
def add_grade_in_arr(knn_stu_crs, gradee_arr):
    for i in range(len(knn_stu_crs)):
        knn_stu_crs[i] = [knn_stu_crs[i][0],gradee_arr[i],knn_stu_crs[i][1]]    
    return knn_stu_crs

#Checking how many electives a student has studied of CS and MG type. Call in action
def cs_mg_electives(exmple ,ecg):
    cs_elects = 0
    mg_elects = 0
    cred_hrs = 0
    
    for i in range(len(ecg)):
        if ecg[i] != -1:
            for j in range(len(exmple)):
                if ecg[i][0] == exmple['Course Title'].iloc[j]:
                    cred_hrs = cred_hrs + float(exmple["Credit Hours"].iloc[j])
                    break
                    
            
    
    for i in range(len(exmple)):
        for j in course_dict.keys():
            if course_dict[j][0] == exmple['Course Title'].iloc[i]:
                if course_dict[j][1] == 0:
                    if course_dict[j][2] == "CS":
                        cs_elects = cs_elects + 1
                        break
                    
                    if course_dict[j][2] == "MG":
                        mg_elects = mg_elects + 1
                        break
                        
    return cred_hrs, cs_elects, mg_elects

In [59]:
#Checking how many electives a student has studied of CS and MG type. Call in action
def cs_mg_electives(exmple):
    cs_elects = 0
    mg_elects = 0
    cred_hrs = 0
    
    for i in range(len(exmple)):
        cred_hrs = cred_hrs + float(exmple["Credit Hours"].iloc[i])
        if course_dict[exmple['Course Title'].iloc[i]][1] == 0:
                
            if course_dict[exmple['Course Title'].iloc[i]][2] == "CS":
                cs_elects = cs_elects + 1 
                    
            if course_dict[exmple['Course Title'].iloc[i]][2] == "MG":
                mg_elects = mg_elects + 1
                        
    return cred_hrs, cs_elects, mg_elects

In [60]:
def updating_credits(cred_hrs, newsem,cs_elects,mg_elects):
    for i in range(len(newsem)):
        cred_hrs = cred_hrs + float(newsem['Credit Hours'].iloc[i])
        if course_dict[newsem['Course Title'].iloc[i]][1] == 0:
                
            if course_dict[newsem['Course Title'].iloc[i]][2] == "CS":
                cs_elects = cs_elects + 1 
                    
            if course_dict[newsem['Course Title'].iloc[i]][2] == "MG":
                mg_elects = mg_elects + 1
    
    return cred_hrs, cs_elects, mg_elects

In [61]:
def degree_complete(crd_hrs,cs_elects, mg_elects):
    if crd_hrs >= 130 and cs_elects == 6 and mg_elects == 4:
        return True 

In [62]:
def unique_in_fcrs(fcrs):
    tmp = []
    for i in range(len(fcrs)):
        if fcrs[i] == 96:
            tmp.append(fcrs[i])
        if fcrs[i] == 97:
            tmp.append(fcrs[i])
        if fcrs[i] not in tmp:
            tmp.append(fcrs[i])
                
    return tmp

In [63]:
def noWarning_musheerSuggestion(ex_cg,student, filtered_courses, musttake_courses,sgpas,avgs,s_c_o, cs_elects, mg_elects, tot_creds):    
    all_terms = student['Semester'].unique()
    semesters = student['Semester'].iloc[len(student)-1]
    if semesters == 0:
        semesters = all_terms[len(all_terms)-2]
        
    if len(str(semesters))>2:
        semesters = all_terms[len(all_terms)-2]
    
        
    if ex['Sr. No'].iloc[0] == 10 or ex['Sr. No'].iloc[0] == 10.0 or ex['Sr. No'].iloc[0] == "10": 
        fcrs = course_suggestionnn(ex, degree_complete, semesters+1, 0)
        
    else:
        df2 = divide_l_df(semesters)
        course_grade1 = all_stu_course_grade(df2)
        colss = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15',
           '16','17','18','19','20','21','22','23','24',
           '25','26','27','28','29','30','31','32','33',
           '34','35','36','37','38','39','40','41','42', 
           '43','44','45','46','47','48','49','50','51', 
           '52','53','54','55','56','57','58','59','60', 
           '61','62','63','64','65','66','67','68','69',
           '70','71','72','73','74','75','76','77','78',
           '79','80','81','82','83','84','85','86','87',
           '88','89','90','91','92','93','94','95','96','97''Student']

        numRows=len(course_grade1)
        df_list = []
        for i in range(numRows):
            df_list.append([])
            for j in range(0, 99):
                df_list[i].append(-1)

        for i in range(numRows):
            for j in range(1,98):
                if (course_grade1[i][j]!=-1):
                    df_list[i][j-1]=float(course_grade1[i][j][1])
            df_list[i][98] = course_grade1[i][0]

        df_ = pd.DataFrame(df_list)

        neigh=KNeighborsClassifier(n_neighbors=3)
        neigh.fit(df_.iloc[:,:98],df_[98])

        for i in range(df_.shape[1]-1):
            df_[i] = df_[i].astype(float)


        gplist = []

        for i in range(len(ex_cg)):
            if ex_cg[i] != -1:
                gplist.append(ex_cg[i][1])
            else:
                gplist.append(-1)

        test = [gplist]

        total_neighbors = neigh.kneighbors(test)[1][0]
        n_stud = {} #list of dataframes of neighboring students
        for i in range(len(total_neighbors)):
            neighbor = df_[98].iloc[total_neighbors[i]]
            n_stud[i] =  isldata[isldata['Sr. No'] == neighbor]
        x = n_stud[1]['Semester']
        x=x.tolist()

        for i in range(len(n_stud)):
            stu_termsN = n_stud[i]["Semester"]
            stu_termsN = stu_termsN.tolist()
            temp_termN = stu_termsN[0]
            termN = 1
            new_termN = []
            for j in range(len(stu_termsN)):
                if "Summer" in stu_termsN[j]:
                    new_termN.append(-1)
                elif temp_termN == stu_termsN[j]:
                    new_termN.append(termN)     
                else:
                    termN = termN + 1
                    temp_termN = stu_termsN[j]
                    new_termN.append(termN)
            n_stud[i]["Semester"]=new_termN

        #selected courses of 4 students for student from previous data
        selected_courses = {}
        for i in range (len(n_stud)):
            selected_courses[i] =n_stud[i].loc[n_stud[i]['Semester']==semesters+1]

        print("Wot sem r u",semesters)
        fcrs = filter_suggestions(student, selected_courses,musttake_courses,filtered_courses,17,s_c_o, cs_elects, mg_elects, degree_template,semesters+1, tot_creds)
        
        
    #fcrs = list(set(fcrs))
    fcrs = unique_in_fcrs(fcrs)
    print('fcrs', fcrs)
    
    
    knn_stu_crs = knn_pred_courses(fcrs,musttake_courses,s_c_o)
    
    if(len(ex["CGPA"].iloc[len(ex)-1])<5):
        gpaaa = float(ex["CGPA"].iloc[len(ex)-1])
    else:
        gpaRange = ex["CGPA"].iloc[len(ex)-1][:4]
        gpaaa = float(gpaRange)
    
    minG_rmve_warning = min_grade_to_remove_warning(knn_stu_crs, gpaaa,ex)
    
    g_a =assign_Grades(fcrs,avgs,ex_cg,minG_rmve_warning)
    
    knn_stu_crs = add_grade_in_arr(knn_stu_crs, g_a)
    
    
    
    rangee = []
    rangee = cgpa_range(knn_stu_crs,rangee)
    mingpa = gpa_calculate(0,rangee,knn_stu_crs)
    maxgpa = gpa_calculate(1,rangee,knn_stu_crs)
    rangee1 = gradePoint_to_letterGrade(rangee)
    musheer_given_courses = musheer_suggestion(knn_stu_crs,rangee1)
    
  
    cgpa_musheer = str(round(mingpa,2)) + " - " + str(round(maxgpa,2))

    new_sem_df_ = pd.DataFrame()
    new_sem_df_ = newSem_df(ex, rangee1,knn_stu_crs,mingpa,maxgpa,sgpas,semesters)
    
    return musheer_given_courses, cgpa_musheer, new_sem_df_

In [64]:
hellp = [21, 32 ,21, 5, 6]
print(hellp)
hellp.remove(21)
print(hellp)

[21, 32, 21, 5, 6]
[32, 21, 5, 6]


In [69]:
hellp = [21, 32 ,21, 5, 6]
hellp1 = [21, 32 ,21, 5, 6]
hellp.append(hellp1)
print(hellp)

[21, 32, 21, 5, 6, [21, 32, 21, 5, 6]]


# --------------------------Functions end here------------------------------------

In [70]:
app=Flask(__name__)
app.secret_key = "super secret key"
selected_courses = []
student_rollno = ""
ex = pd.DataFrame()
st = pd.DataFrame()
newsem = pd.DataFrame()
cgpa_musheer=0
can_take_courses = []
can_take_courses_string = []
can_take_courses_new = []
can_take_courses_new_string = []
can_take_courses_repeat = []
can_take_courses_repeat_string = []
must_take_courses = []
must_take_courses_string = []
ecg = []
sem_course_offrng = pd.DataFrame()
avgs = []
creds_earned = 0
cs_elects = 0
mg_elects = 0

In [71]:
@app.route("/")
def login():
    return render_template('index.html')

@app.route('/action', methods=['GET', 'POST'])
def action():
    if request.method == "POST":
        rollno = request.form['email']
        global st
        global ecg
        global ex
        global cgpa_musheer
        global can_take_courses
        global can_take_courses_string
        global can_take_courses_new
        global can_take_courses_new_string
        global can_take_courses_repeat
        global can_take_courses_repeat_string
        global must_take_courses
        global must_take_courses_string
        global crs_type_arr
        global avgs
        global creds_earned
        global cs_elects
        global mg_elects
        
        ex = isldata[isldata['Sr. No'] == int(rollno)]
        #rand = ex_stu_course_grade(ex)
        
        ex = semstoNumbers(ex)
        creds_earned, cs_elects, mg_elects = cs_mg_electives(ex)
        print("Creds in  action:", creds_earned)
        print("cs in action:", cs_elects)
        print("mg in action",mg_elects)
    
        
        
        crs_type_arr = ex_stu_course_type(ex)
        avgs = ex_course_avg(crs_type_arr)
        print("avgs", avgs)

        cgpa_musheer=0
        list_objects = []
        groupby_object = ex.groupby('Semester', sort=False)
        sem = []
        cgpa = []
        sgpa = []
        war = []
        for group in groupby_object.groups:
            list_objects.append(groupby_object.get_group(group)[['Course Code','Course Title','Credit Hours','Grade']].to_html(classes='table table-head-bg-info table-striped Transcript'))
   
            sem.append(groupby_object.get_group(group)['Semester'].iloc[0])
            cgpa.append(groupby_object.get_group(group)['CGPA'].iloc[0])
            sgpa.append(groupby_object.get_group(group)['SGPA'].iloc[0])
            war.append(groupby_object.get_group(group)['Warning'].iloc[0])
               
    return render_template("transcript.html",tables=zip(list_objects, cgpa, sgpa, sem, war), titles=ex.columns.values, rolln=rollno)


@app.route('/addSem', methods=['GET', 'POST'])
def addSem():
    if request.method == "POST":
        global st
        global ecg
        global newsem
        global ex
        global cgpa_musheer
        global can_take_courses
        global can_take_courses_string
        global can_take_courses_new
        global can_take_courses_new_string
        global can_take_courses_repeat
        global can_take_courses_repeat_string
        global must_take_courses
        global must_take_courses_string
        global creds_earned
        global cs_elects
        global mg_elects
        global crs_type_arr
        
        rollno=ex['Sr. No'].iloc[0]
        #student_rollno =rollno
      
        creds_earned, cs_elects, mg_elects = updating_credits(creds_earned, newsem, cs_elects, mg_elects)
        #ex, can_take_courses, can_take_courses_string, can_take_courses_new, can_take_courses_new_string, can_take_courses_repeat, can_take_courses_repeat_string, must_take_courses, must_take_courses_string = example_student(ex)
        
        print("Cred earned:",creds_earned)
        print("CS elects:", cs_elects)
        print("MG elects", mg_elects)
        
        if degree_complete(creds_earned,cs_elects, mg_elects) == True:
            flash("You have reached the credit limit for completion of degree.")
            print("U HAVE COMPLETED")
        
        cgpa_musheer=0
        list_objects = []
        list_objects1 = []
        groupby_object = ex.groupby('Semester', sort = False)
        sem = []
        cgpa = []
        sgpa = []
        war = []
        sem1 = []
        cgpa1 = []
        sgpa1 = []
        war1 = []
        
        ex = crsTitlestoNames(ex)
       
       
        
        
        for group in groupby_object.groups:
            list_objects.append(groupby_object.get_group(group)[['Course Code','Course Title','Credit Hours','Grade']].to_html(classes='table table-head-bg-info table-striped Transcript'))

            sem.append(groupby_object.get_group(group)['Semester'].iloc[0])
            cgpa.append(groupby_object.get_group(group)['CGPA'].iloc[0])
            sgpa.append(groupby_object.get_group(group)['SGPA'].iloc[0])
            war.append(groupby_object.get_group(group)['Warning'].iloc[0])
        

       
        groupby_object = newsem.groupby('Semester', sort = False)
        for group in groupby_object.groups:
            list_objects1.append(groupby_object.get_group(group)[['Course Code','Course Title','Credit Hours','Grade']].to_html(classes='table table-head-bg-success table-striped Transcript'))

            sem1.append(newsem['Semester'].iloc[0])
            cgpa1.append(groupby_object.get_group(group)['CGPA'].iloc[0])
            sgpa1.append(groupby_object.get_group(group)['SGPA'].iloc[0])
            war1.append(groupby_object.get_group(group)['Warning'].iloc[0])
                
       
        ex = ex.append(newsem)
        
          
    return render_template("transcript.html",tables=zip(list_objects, cgpa, sgpa, sem, war),tables1=zip(list_objects1, cgpa1, sgpa1, sem1, war1), titles=ex.columns.values, rolln=rollno)

    
@app.route("/home",methods=['GET', 'POST'])
def home():
    global st
    global ecg
    global ex
    global cgpa_musheer
    global can_take_courses
    global can_take_courses_string
    global can_take_courses_new
    global can_take_courses_new_string
    global can_take_courses_repeat
    global can_take_courses_repeat_string
    global must_take_courses
    global must_take_courses_string
    global sem_course_offrng
    rollno=ex['Sr. No'].iloc[0]
    #print (ex)
    ex = crsTitlestoNums(ex)
    ecg, ex, can_take_courses, can_take_courses_string, can_take_courses_new, can_take_courses_new_string, can_take_courses_repeat, can_take_courses_repeat_string, must_take_courses, must_take_courses_string,sem_course_offrng = example_student(ex)
        
        
    c = ex['CGPA'].iloc[len(ex)-1]
    s = ex['SGPA'].iloc[len(ex)-1]
    
    if len(c) >4 or len(s)>4:
        c = c[0] + c[1] +c[2] + c[3]
        s = s[0] + s[1] +s[2] + s[3]
        
    
    print("type",type(ex["Warning"].iloc[len(ex)-1]))
    if float(c)<=1 or float(s)<=1:
        return render_template('lessgpa.html',cantake = can_take_courses_string, musttake = must_take_courses_string, rollno = student_rollno)
    
    else:
        if ex["Warning"].iloc[len(ex)-1] == "2.0" or ex["Warning"].iloc[len(ex)-1] == "2" :
            return render_template('home.html', c=cgpa_musheer, cantake = can_take_courses_repeat_string, musttake = must_take_courses_string, rollno = student_rollno)

        else:    
            return render_template('home.html', c=cgpa_musheer, cantake = can_take_courses_string, musttake = must_take_courses_string, rollno = student_rollno)

    
@app.route("/askSughome",methods=['GET', 'POST'])
def askSughome():
    global st
    global ecg
    global ex
    global cgpa_musheer
    global can_take_courses
    global can_take_courses_string
    global can_take_courses_new
    global can_take_courses_new_string
    global can_take_courses_repeat
    global can_take_courses_repeat_string
    global must_take_courses
    global must_take_courses_string
    global sem_course_offrng
    rollno=ex['Sr. No'].iloc[0]
    #print (ex)
    #ex = crsTitlestoNums(ex)
    ecg, ex, can_take_courses, can_take_courses_string, can_take_courses_new, can_take_courses_new_string, can_take_courses_repeat, can_take_courses_repeat_string, must_take_courses, must_take_courses_string,sem_course_offrng = example_student(ex)
    
    c = ex['CGPA'].iloc[len(ex)-1]
    s = ex['SGPA'].iloc[len(ex)-1]
    
    if ex["Warning"].iloc[len(ex)-1] == "2.0" or ex["Warning"].iloc[len(ex)-1] == "2":
        return render_template('askSughome.html', c=cgpa_musheer, cantake = can_take_courses_repeat_string, musttake = must_take_courses_string, rollno = student_rollno)

    else:
        return render_template('askSughome.html', c=cgpa_musheer, cantake = can_take_courses_string, musttake = must_take_courses_string, rollno = student_rollno)

@app.route("/makeplan",methods=['GET', 'POST'])
def makeplan():
    global st
    global ecg
    global ex
    global cgpa_musheer
    global can_take_courses
    global can_take_courses_string
    global can_take_courses_new
    global can_take_courses_new_string
    global can_take_courses_repeat
    global can_take_courses_repeat_string
    global must_take_courses
    global must_take_courses_string
    global sem_course_offrng
    rollno=ex['Sr. No'].iloc[0]
    #print (ex)
    #ex = crsTitlestoNums(ex)
    ecg, ex, can_take_courses, can_take_courses_string, can_take_courses_new, can_take_courses_new_string, can_take_courses_repeat, can_take_courses_repeat_string, must_take_courses, must_take_courses_string,sem_course_offrng = example_student(ex)
    
    c = ex['CGPA'].iloc[len(ex)-1]
    s = ex['SGPA'].iloc[len(ex)-1]
    
    if ex["Warning"].iloc[len(ex)-1] == "2.0" or ex["Warning"].iloc[len(ex)-1] == "2":
        return render_template('makeplan.html', c=cgpa_musheer, cantake = can_take_courses_repeat_string, musttake = must_take_courses_string, rollno = student_rollno)

    
    else:
        return render_template('makeplan.html', c=cgpa_musheer, cantake = can_take_courses_string, musttake = must_take_courses_string, rollno = student_rollno)

@app.route('/submit', methods=['GET', 'POST'])
def submit():
    if request.method == 'POST':
        global ex
        global st
        global newsem
        global sem_course_offrng
        global avgs
        print("url")
        print(request.url)
        checkboxes2 = []
        checkboxes3 = []
        
        for can in can_take_courses_string:
            checkbox = request.form.get('chkbox' + can)
            checkboxes2.append(checkbox)
        
        for must in must_take_courses_string:
            checkbox = request.form.get('chkbox' + must)
            checkboxes3.append(checkbox)

        newlist = []
        stu_choose_self = []
        for i in range(len(checkboxes2)):
            if checkboxes2[i]=='on':
                stu_choose_self.append(can_take_courses_string[i])
                
        
        for i in range(len(checkboxes3)):
            stu_choose_self.append(must_take_courses_string[i])
        
        crdt_limit = 0
        
        for i in range (len(stu_choose_self)):
            stu_choose_self[i] = course_dict[stu_choose_self[i]][0] 
        
        for i in range(len(sem_course_offrng)):
            for j in range(len(stu_choose_self)):
                if int(sem_course_offrng['Course Title'].iloc[i]) == stu_choose_self[j]:
                    crdt_limit = crdt_limit + sem_course_offrng['Credit Hours'].iloc[i]
                    
        print(crdt_limit)
            
        g_a = assign_Grades(stu_choose_self,avgs,ecg,2.0)
        stu_choose_self = student_choice_courses(stu_choose_self,sem_course_offrng,g_a)            
        
        if (len(ex['CGPA'].iloc[len(ex)-1])<5):
            stu_cgpa = float(ex['CGPA'].iloc[len(ex)-1])
        else:
            gpaRange = ex['CGPA'].iloc[len(ex)-1][:4] #picking the min value of cgpa
            #print (gpaRange)
            stu_cgpa = float(gpaRange)
        
        exS_crses =[]
        for i in range(len(ex)):
            exS_crses.append(ex['Course Title'].iloc[i])
        len(exS_crses)
    
        #stu_choose_self = student_choice_gpa_range_pred(stu_choose_self,stu_cgpa,ex)
        rangee1= []
        rangee1 = cgpa_range(stu_choose_self,rangee1)
    
        mingpa1 = gpa_calculate(0,rangee1,stu_choose_self)
        maxgpa1 = gpa_calculate(1,rangee1,stu_choose_self)
        
        rangee2 = gradePoint_to_letterGrade(rangee1)
        
        gr = ex.groupby('Semester', sort=False)
        sgpas = []
        semss = 0
        for group in gr.groups:
            semss = semss + 1
            if len(gr.get_group(group)['SGPA'].iloc[0])<5:
                sgpas.append(gr.get_group(group)['SGPA'].iloc[0])
            else:
                sgpas.append(gr.get_group(group)['SGPA'].iloc[0][:4])
         
        self_given_courses = musheer_suggestion(stu_choose_self,rangee2)
        
        cgpa_selected = str(round(mingpa1,2)) + " - " + str(round(maxgpa1,2))
        newsem = newSem_df(ex, rangee2,stu_choose_self,mingpa1,maxgpa1,sgpas,semss)
        
        if(crdt_limit>18):
            flash("credit limit exceeded! Please choose the courses again.","danger")
            return redirect("http://127.0.0.1:5000/makeplan")
        else:
            flash("Courses Selected","success")
        #-----------------------------------------------------------------------
        if ex["Warning"].iloc[len(ex)-1] == "2.0" or ex["Warning"].iloc[len(ex)-1] == "2":
            return render_template('makeplan.html',c=cgpa_selected, cantake = can_take_courses_repeat_string, musttake = must_take_courses_string, chosen = [self_given_courses.to_html(classes = 'table table-head-bg-info table-striped CoursesSelf')],newsem = newsem)
    
        
        else:
            return render_template('makeplan.html',c=cgpa_selected, cantake = can_take_courses_string, musttake = must_take_courses_string, chosen = [self_given_courses.to_html(classes = 'table table-head-bg-info table-striped CoursesSelf')],newsem = newsem)
    
    
@app.route('/ask_suggestion', methods=['GET', 'POST'])
def ask_suggestion():
    if request.method == 'POST':
        global ecg
        global st
        global ex
        global newsem
        global cgpa_musheer
        global can_take_courses
        global can_take_courses_string
        global can_take_courses_new
        global can_take_courses_new_string
        global can_take_courses_repeat
        global can_take_courses_repeat_string
        global must_take_courses
        global must_take_courses_string
        global crs_type_arr
        global avgs
        global sem_course_offrng
        global mg_elects
        global cs_elects
        global creds_earned
        #print (can_take_courses)
        
        gr = ex.groupby('Semester', sort=False)
        sgpas = []
        for group in gr.groups:
            #if len(gr.get_group(group)['SGPA'].iloc[0])<5:
            sgpas.append(gr.get_group(group)['SGPA'].iloc[0])
            #else:
             #   sgpas.append(gr.get_group(group)['SGPA'].iloc[0][:4])
         
        if ex["Warning"].iloc[len(ex)-1] == "2.0" or ex["Warning"].iloc[len(ex)-1] == "2":
            musheer_given_courses,cgpa_musheer,newsem, maxxx = warning2_musheerSuggestion(ex, can_take_courses_repeat, must_take_courses, sem_course_offrng,ecg,sgpas)
            if maxxx < 2.0:
                flash("Your GPA was below 2. Please contact your Advisor","danger")
            
        else:
            musheer_given_courses,cgpa_musheer,newsem = noWarning_musheerSuggestion(ecg,ex, can_take_courses, must_take_courses,sgpas,avgs,sem_course_offrng, cs_elects, mg_elects, creds_earned)
        #print('musheeer',musheer_given_courses)
        
        
        if ex["Warning"].iloc[len(ex)-1] == "2.0" or ex["Warning"].iloc[len(ex)-1] == "2":
            return render_template('askSughome.html', c=cgpa_musheer, cantake = can_take_courses_repeat_string, musttake = must_take_courses_string, selected = [musheer_given_courses.to_html(classes = 'table table-head-bg-info table-striped Courses')])
        
        
        else:
            return render_template('askSughome.html', c=cgpa_musheer, cantake = can_take_courses_string, musttake = must_take_courses_string, selected = [musheer_given_courses.to_html(classes = 'table table-head-bg-info table-striped Courses')])
        
@app.route('/to_endPage', methods=['GET', 'POST'])
def askAdvisor():
    if request.method == 'POST':
        #contentStr=''
        global ex
        
        return render_template('endPage.html')
    
@app.route('/sendpls', methods = ['POST'])
def postpls():
    data = request.form['mail']
    print ("Hello world!")
    print (data)

    if request.method == 'POST':
        #contentStr=''
        global ex
        try:
            #initializing the server connection
            yag = yagmail.SMTP(user='askadvisor.musheer@gmail.com', password='abcd@1234')
            ex.to_csv('StudentPlan.csv')
            print("rollnummmmm")
            print(ex['Sr. No'].iloc[0])
            sub = 'Semester plan for '+str(ex['Sr. No'].iloc[0])
            print(sub)
            yag.send(to='mushjerry@gmail.com', subject=sub, contents=[data,"StudentPlan.csv"])
            print("Email sent successfully")
            flash("Your email along with semester plan has been sent to the Advisor!","success")
        except:
            print("Error, email was not sent")
            
            
        return render_template('endPage.html')

In [72]:
if __name__=='__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Jun/2020 09:46:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2020 09:46:04] "GET /static/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:46:04] "GET /static/fonts/font-awesome-4.7.0/css/font-awesome.min.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:46:04] "GET /static/fonts/Linearicons-Free-v1.0.0/icon-font.min.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:46:04] "GET /static/vendor/animate/animate.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:46:04] "GET /static/vendor/css-hamburgers/hamburgers.min.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:46:04] "GET /static/vendor/animsition/css/animsition.min.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:46:04] "GET /static/vendor/select2/select2.min.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:46:04] "GET /static/vendor/daterangepicker/daterangepicker.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:46

Creds in  action: 100.0
cs in action: 3
mg in action 2
avgs [3.2857142857142856, 3.28, 3.1675, 2.335, 2.835, 3.0, 3.3340000000000005, 3.33]


127.0.0.1 - - [11/Jun/2020 09:46:33] "POST /action HTTP/1.1" 200 -
M:\Python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Ab is semester ki cheez dhundani hai: 7
type <class 'str'>


127.0.0.1 - - [11/Jun/2020 09:47:13] "POST /home HTTP/1.1" 200 -
M:\Python\lib\site-packages\sklearn\utils\validation.py:563: FutureWarning: Beginning in version 0.22, arrays of bytes/strings will be converted to decimal numbers if dtype='numeric'. It is recommended that you convert the array to a float dtype before using it in scikit-learn, for example by using your_array = your_array.astype(np.float64).
  FutureWarning)
M:\Python\lib\site-packages\ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Wot sem r u 6
Credits in normal: 15
replacee []
mg elects 2
mg count 0
suggestions in filter_sugs [54, 62, 96, 96, 56, 97]
fcrs [54, 62, 96, 96, 56, 97]
[56, 3.0] 3.0
[54, 3.0] 3.0
[62, 3.0] 3.0
[96, 3.0] 3.0
[96, 3.0] 3.0
[97, 3.0] 3.0
stu choose self in min grade warning: [[54, 3.0], [62, 3.0], [96, 3.0], [96, 3.0], [56, 3.0], [97, 3.0]]
knn crsesss in musheer suggestion: [[54, 3.33, 3.0], [62, 3.33, 3.0], [96, 3.33, 3.0], [96, 3.33, 3.0], [56, 3.33, 3.0], [97, 3.0, 3.0]]


127.0.0.1 - - [11/Jun/2020 09:47:22] "POST /ask_suggestion HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2020 09:47:30] "GET /? HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2020 09:47:30] "GET /static/fonts/Linearicons-Free-v1.0.0/icon-font.min.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:47:30] "GET /static/vendor/animate/animate.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:47:30] "GET /static/vendor/css-hamburgers/hamburgers.min.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:47:30] "GET /static/vendor/animsition/css/animsition.min.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:47:30] "GET /static/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:47:30] "GET /static/vendor/daterangepicker/daterangepicker.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:47:30] "GET /static/fonts/font-awesome-4.7.0/css/font-awesome.min.css HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2020 09:47:30] "GET /static/vendor/select2/select2.min.css HTTP/1.1" 404 -
127.0.0.1 - - [1

#-----------------------------------------------------------------------------------